# Pythonで沖縄電力の需給実績情報の読み込みとプロット
ver 0.11

&copy; S. Harada

2023.02.14 最終更新

## 目的
電力市場の自由化に伴い，多くの情報が公開されるようになっている。これらは非常に有用なデータであるが，数千行以上が当たり前のデータであり，Excelなどの表計算ソフトで処理することは可能であるが，面倒である。
ここではPythonにより沖縄電力の需給実績情報のデータを処理しプロットする方法を紹介する。

## 必要な環境
### Jupyter LabとPythonモジュール
このファイルはJupyter Labから利用することを想定している。
Jupyter Labは自分のPython環境に合わせてインストールすること。

またpanadasとbokehライブラリを使用しているため自分の利用するPythonのパッケージ管理システム（pipやcondaなど）を利用してインストールしておくこと。

### データファイルのダウンロード
沖縄電力の公表している需給関連情報は以下のURLからダウンロードできる。
ここでは，2016年のデータを2016.csvという名前でこのファイルと同じディレクトリに保存していることを前提とする。

沖縄電力需給関連情報(需給実績)の公表 : [https://www.okiden.co.jp/business-support/service/supply-and-demand/](https://www.okiden.co.jp/business-support/service/supply-and-demand/)

なお，Pythonにより自動的にダウンロードする仕組みを作ることは簡単であるが，ファイル全体を実行した時に同じファイルを繰り返しダウンロードすることを避けるために，ここでは別の手段でダウンロードしてあることを前提とした。

### JupyterLabの操作
* このファイルに含まれる全てのプログラムを実行するには上部のRunメニューからRun All Cellsを選ぶ。
* このファイルはipyhotn notebookと呼ばれる形式でファイルはMarkdwonセルとCodeセルから構成される。
* Markdownセルは，このセルのように文章・数式など説明のためのセルである。
* Markdownセルをダブルクリックすると編集モードになり，シフトキーを押した状態でEnterキーを押すと書式の反映された表示モードになる。
* Codeセルはプログラムを入力するためのセルであり，セルの左側上部に [1] のように数字が表示されている。この数字は，この環境を立ち上げてから何回目に実行されたセルかを示している。
* Codeセルの下にはプログラムの実行により出力された結果が表示される。
* Codeセルを個別に実行するにはコードセルを選択し，シフトキーを押した状態でEnterキーを押す。



### 必要なモジュールの読み込み

このプログラムではデータを操作するために，datetimeというPythonの標準ライブラリのモジュール，pandasという統計などに便利なデータ構造のための外部モジュール，bokehというグラフを描くためのモジュールを使用する。

なお，プログラムの各行の`#`以降はコメントとして扱われプログラムの動作には影響しない。

In [18]:
# 必要なモジュールをインポート
from datetime import date #日付処理用モジュールを読み込み
import pandas as pd #データ操作用モジュールを読み込み

# 以下はグラフを描かなければ不要
from bokeh.plotting import figure, show, output_notebook #グラフ表示モジュールを読み込み
from bokeh.palettes import Category10 as palette #色のパレットを読み込み
output_notebook() 


Loading BokehJS ...

### データファイルの読み込みとデータの整理
以下のプログラムは 2016.csv という2016年度の需給実績データファイルを読み込み，処理し易いように整理する。

まずpd.read_csvでデータを読み込み，データフレームと呼ばれる表のような形式のフォーマットで変数`df_loadokinawa`に代入する。

元のファイルの見出し行（ヘッダー）が複数行にまたがり扱い難いので新規に見出し行を設定している。

太陽光の影響を調べるために，太陽光以外の発電所が担うべき正味需要（需要実績-太陽光の発電量）の列を追加している。

集計し易いように，日時の列，年月日，年月，曜日を追加し，1時間ごとのデータなので時刻を01:00の形式から時の数字だけを抜き出した形式に変換している。

これから`df_loadokinawa`を操作することで，集計を行う。

In [2]:
# 沖縄電力の需給実績をダウンロードして左のファイル表示欄にドラッグ
# 電力のデータは MWh である
# filename変数の文字列をファイル名に合わせる
filename="2016.csv"
#
#
df_loadokinawa=pd.read_csv(filename,encoding='cp932',header=[4],skiprows=[5,6])
# 見出し行（ヘッダー）を整理
new_columns=["日付","時刻","需要実績","空列0","火力","水力","バイオマス","太陽光","太陽光制御","風力","風力制御","発電合計"]

# 年によって列数が違うので最後列に空の列がある場合に削除
df_loadokinawa.drop(df_loadokinawa.columns[range(len(new_columns),len(df_loadokinawa.columns.values))],
                    axis=1,inplace=True)
# 整理した見出し行に変更
df_loadokinawa.columns=new_columns
# 途中にある無駄な空の列を削除
df_loadokinawa.drop("空列0",axis=1,inplace=True)

# Pythonで扱い易いように文字列の日付情報と時刻情報から日時の列を生成
df_loadokinawa["日時"]= pd.to_datetime(df_loadokinawa["日付"]+"/"+df_loadokinawa["時刻"], format='%Y/%m/%d/%H:%M')
# 太陽光を除いた他の発電が担うべき正味需要を計算
df_loadokinawa["正味需要"]=df_loadokinawa["需要実績"]-df_loadokinawa["太陽光"]
# 日時情報から年月日だけ抜き出した列を生成（日単位の処理を簡単にするため）
df_loadokinawa["年月日"]=df_loadokinawa["日時"].dt.strftime("%Y-%m-%d").astype("datetime64")
# 日時情報から年月だけ抜き出した列を生成（月単位の処理を簡単にするため，年度内の並べ替えで順序がおかしくならないように年月で扱う）
df_loadokinawa["年月"]=df_loadokinawa["日時"].dt.strftime("%Y-%m")
# 日時情報から曜日の列を生成（曜日単位の処理を簡単にするため）
df_loadokinawa["曜日"]=df_loadokinawa["日時"].dt.day_name()
# 毎時0分の情報のみなので時刻は時のみを抜き出す
df_loadokinawa["時刻"]=df_loadokinawa["日時"].dt.hour

結果を表示するには`display`関数に表示したい変数を渡す。8760行のデータのため，途中は省略されて表示されている。

In [3]:
display(df_loadokinawa)

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,年月,曜日
0,2016/4/1,0,647,643,1,2,0,0,2,0,647,2016-04-01 00:00:00,647,2016-04-01,2016-04,Friday
1,2016/4/1,1,630,625,1,2,0,0,3,0,630,2016-04-01 01:00:00,630,2016-04-01,2016-04,Friday
2,2016/4/1,2,644,640,1,2,0,0,2,0,644,2016-04-01 02:00:00,644,2016-04-01,2016-04,Friday
3,2016/4/1,3,663,658,1,2,0,0,2,0,663,2016-04-01 03:00:00,663,2016-04-01,2016-04,Friday
4,2016/4/1,4,657,652,1,2,0,0,3,0,657,2016-04-01 04:00:00,657,2016-04-01,2016-04,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017/3/31,19,881,861,1,3,0,0,14,0,881,2017-03-31 19:00:00,881,2017-03-31,2017-03,Friday
8756,2017/3/31,20,849,828,1,3,0,0,15,0,849,2017-03-31 20:00:00,849,2017-03-31,2017-03,Friday
8757,2017/3/31,21,804,785,1,3,0,0,14,0,804,2017-03-31 21:00:00,804,2017-03-31,2017-03,Friday
8758,2017/3/31,22,752,733,1,3,0,0,13,0,752,2017-03-31 22:00:00,752,2017-03-31,2017-03,Friday


なお，Codeセルの最後の行では`display`を省略して変数名だけでも同じ結果が得られるが，表示することを明示するために敢えて使っている。

### データの集計

#### 日毎の合計の計算

1時間ごとのデータを日毎のデータに集計するために以下のプログラムを使う。`.groupby("年月日")`で年月日が同じもの同士に操作をすることが可能になる。操作として合計`.sum()`を適用する。これにより，同じ日のデータの合計が計算される。
ただし，月や時刻の合計は意味のない情報のため，`[["需要実績","太陽光","正味需要"]]`により和を計算したい列を指定している。
得られた結果を`df_loadokinawa_daily_sum`という変数に代入し，表示する。

In [4]:
# 1日ごとの合計を出したければgroupbyメソッドで年月日のグループごとに取り出しsumで総和を取る
# ["需要実績","太陽光","正味需要"]のところにある列だけを抜き出して合計をとる
df_loadokinawa_daily_sum=df_loadokinawa.groupby("年月日")[["需要実績","太陽光","正味需要"]].sum()
display(df_loadokinawa_daily_sum)

,需要実績,太陽光,正味需要
年月日,,,
2016-04-01,18328,1660,16668
2016-04-02,18560,1624,16936
2016-04-03,18148,1577,16571
2016-04-04,19692,535,19157
2016-04-05,20018,1036,18982
...,...,...,...
2017-03-27,18463,1879,16584
2017-03-28,18420,1901,16519
2017-03-29,18173,1313,16860


#### CSV形式での保存

集計結果をExcelなどの表計算ソフトで読めるようにCSV形式（データの各列がカンマで区切られた形式）で保存することができる。
これはデータフレームに`.to_csv`を付けてファイル名など必要な情報を渡すことで可能になる。下の例では`df_loadokinawa_daily_sum`という上で求めた日毎の合計を集計した結果を loadokinawa_daily_sum.csv というファイル名で保存する。実行すると左側のファイルブラウザにファイルが現れる。

`encoding='cp932'`はダブルクリックでMicrosoft Excelで開いた時に文字化けを回避するために文字コードを日本語シフトJISで保存するための設定である。今時はUTF-8を使用することが普通になっているので，Excel以外で開くことを目的とする場合は，`encoding='cp932'`のない最下行の行頭の#を削除して有効にしてUTF-8のファイルを生成することを勧める。

なお，Excelでも新規ファイルを作成し，「データ」タブの「データの取得」の「テキスト/CSVから」をクリックして操作すると文字コードを指定して取り込むことができUTF-8のファイルも取り込むことが可能です。

In [5]:
# Excelで読めるようにCSV形式で保存することが可能
# Excelでダブルクリックで開いた時に文字化けしないように encoding='cp932' を指定
# 
df_loadokinawa_daily_sum.to_csv("loadokinawa_daily_sum.csv",encoding='cp932')
#
# Excelに読み込ませないのであれば下の行頭の#を削除して使う
#df_loadokinawa_daily_sum.to_csv("loadokinawa_daily_sum.csv")


#### 日毎の最大値の計算

1日の最大値を計算することも同様の方法で可能になる。`.groupby("年月日")`で年月日が同じもの同士を一つのグループとし，操作として最大値`.max()`を適用する。これにより，1日の最大値が計算される。
ただし，月や時刻の合計は意味のない情報のため，`[["需要実績","太陽光","正味需要"]]`により和を計算したい列を指定している。
得られた結果を`df_loadokinawa_daily_max`という変数に代入し，表示する。

なお，最大値の他に最小`.min()`や平均`.mean()`，標準偏差`.std()`も計算可能であるが，太陽光は昼間のみしか発電しないため，あまり意味がない数字となる。

In [6]:
# 1日の最大値を出したければgroupbyメソッドで年月日のグループごとに取り出しmaxを渡す
df_loadokinawa_daily_max=df_loadokinawa.groupby("年月日")[["需要実績","太陽光","正味需要"]].max()
display(df_loadokinawa_daily_max)

,需要実績,太陽光,正味需要
年月日,,,
2016-04-01,863,230,863
2016-04-02,884,233,884
2016-04-03,890,223,890
2016-04-04,945,79,920
2016-04-05,963,154,957
...,...,...,...
2017-03-27,864,256,864
2017-03-28,867,253,867
2017-03-29,863,234,863


なお，`idxmax()`を用いることで最大値をとる行の位置を調べることも可能である。

In [7]:
df_loadokinawa_daily_idxmax=df_loadokinawa.groupby("年月日")[["需要実績","太陽光","正味需要"]].idxmax()
display(df_loadokinawa_daily_idxmax)

,需要実績,太陽光,正味需要
年月日,,,
2016-04-01,19,12,19
2016-04-02,43,36,43
2016-04-03,67,60,67
2016-04-04,85,83,91
2016-04-05,110,110,115
...,...,...,...
2017-03-27,8659,8652,8659
2017-03-28,8683,8676,8683
2017-03-29,8707,8700,8707


上の結果は，4月1日は行19（Pythonは0から数えるので20行目）に需要実績の最大値，行12に太陽光の最大値，行19に正味需要の最大値があることを示している。実際に確認してみよう。実際に最初の24行分を`.head(24)`で表示させて確認してみよう。

In [8]:
display(df_loadokinawa.head(24))

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,年月,曜日
0,2016/4/1,0,647,643,1,2,0,0,2,0,647,2016-04-01 00:00:00,647,2016-04-01,2016-04,Friday
1,2016/4/1,1,630,625,1,2,0,0,3,0,630,2016-04-01 01:00:00,630,2016-04-01,2016-04,Friday
2,2016/4/1,2,644,640,1,2,0,0,2,0,644,2016-04-01 02:00:00,644,2016-04-01,2016-04,Friday
3,2016/4/1,3,663,658,1,2,0,0,2,0,663,2016-04-01 03:00:00,663,2016-04-01,2016-04,Friday
4,2016/4/1,4,657,652,1,2,0,0,3,0,657,2016-04-01 04:00:00,657,2016-04-01,2016-04,Friday
5,2016/4/1,5,635,630,1,2,0,0,3,0,635,2016-04-01 05:00:00,635,2016-04-01,2016-04,Friday
6,2016/4/1,6,653,645,1,2,5,0,1,0,653,2016-04-01 06:00:00,648,2016-04-01,2016-04,Friday
7,2016/4/1,7,715,677,1,2,34,0,1,0,715,2016-04-01 07:00:00,681,2016-04-01,2016-04,Friday
8,2016/4/1,8,786,682,1,2,99,0,2,0,786,2016-04-01 08:00:00,687,2016-04-01,2016-04,Friday
9,2016/4/1,9,818,663,1,2,152,0,2,0,818,2016-04-01 09:00:00,666,2016-04-01,2016-04,Friday


なお，特定の行だけを表示させるには以下のように`iloc`を適用する。

In [9]:
display(df_loadokinawa.iloc[[12,19]])

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,年月,曜日
12,2016/4/1,12,836,602,1,2,230,0,2,0,836,2016-04-01 12:00:00,606,2016-04-01,2016-04,Friday
19,2016/4/1,19,863,861,1,2,0,0,1,0,863,2016-04-01 19:00:00,863,2016-04-01,2016-04,Friday


#### 月毎の合計の計算

月毎の合計を計算することも同様の方法で可能になる。`.groupby("年月")`で月が同じもの同士を一つのグループとし，操作として合計`.sum()`を適用する。これにより，一月の合計が計算される。
ただし，月や時刻の合計は意味のない情報のため，`[["需要実績","太陽光","正味需要"]]`により和を計算したい列を指定している。
得られた結果を`df_loadokinawa_monthly_sum`という変数に代入し，表示する。

In [10]:
# 月ごとも同様に
df_loadokinawa_monthly_sum=df_loadokinawa.groupby("年月")[["需要実績","太陽光","正味需要"]].sum()
display(df_loadokinawa_monthly_sum)

,需要実績,太陽光,正味需要
年月,,,
2016-04,602059,35916,566143
2016-05,697359,37867,659492
2016-06,796932,37331,759601
2016-07,884861,42364,842497
2016-08,869261,40489,828772
2016-09,788481,33091,755390
2016-10,778369,34054,744315
2016-11,595858,26674,569184
2016-12,581289,24215,557074


#### 年間合計の計算
年間の合計は月毎の合計の合計となるので月毎の結果に合計`.sum()`を適用することで得られる。

In [11]:
# 月ごとの合計から年間の合計を算出
df_loadokinawa_yearly_sum=df_loadokinawa_monthly_sum.sum()
display(df_loadokinawa_yearly_sum)

需要実績    8285756
太陽光      387009
正味需要    7898747
dtype: int64

### 特定のデータの抽出

これまでにデータの集計方法を説明したが，一部のデータだけに詳しく見たい場合がある。
dfから始まる名前として定義した変数は全てデータフレームと呼ばれる表のような構造を持っている。これは`.query`という操作を適用することで一部だけを簡単に抜き出すことが出来る。

たとえば2016年8月1日のデータだけ，取り出す例を見てみる。日付は"2016-08-01"のように月と日を二桁で表現する。
年月日の列が"2016-08-01"と一致するデータを取り出すためのプログラムは以下のようになる。

条件式の中の`==`は等しいことを意味する。引用符で囲まれた文字列の中で変数`selected_day`の値を使うために条件式では`@selected_day`と`@`を付けている。これで年月日の列が2016-08-01に等しいという意味となる。

In [12]:
selected_day="2016-08-01"
display(df_loadokinawa.query('年月日 == @selected_day'))

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,年月,曜日
2928,2016/8/1,0,1008,1005,1,2,0,0,0,0,1008,2016-08-01 00:00:00,1008,2016-08-01,2016-08,Monday
2929,2016/8/1,1,946,943,1,2,0,0,0,0,946,2016-08-01 01:00:00,946,2016-08-01,2016-08,Monday
2930,2016/8/1,2,933,930,1,2,0,0,0,0,933,2016-08-01 02:00:00,933,2016-08-01,2016-08,Monday
2931,2016/8/1,3,925,922,1,2,0,0,0,0,925,2016-08-01 03:00:00,925,2016-08-01,2016-08,Monday
2932,2016/8/1,4,916,913,1,2,0,0,0,0,916,2016-08-01 04:00:00,916,2016-08-01,2016-08,Monday
2933,2016/8/1,5,927,924,1,2,0,0,0,0,927,2016-08-01 05:00:00,927,2016-08-01,2016-08,Monday
2934,2016/8/1,6,955,938,1,2,14,0,0,0,955,2016-08-01 06:00:00,941,2016-08-01,2016-08,Monday
2935,2016/8/1,7,1073,1012,1,2,58,0,0,0,1073,2016-08-01 07:00:00,1015,2016-08-01,2016-08,Monday
2936,2016/8/1,8,1206,1122,1,2,81,0,0,0,1206,2016-08-01 08:00:00,1125,2016-08-01,2016-08,Monday
2937,2016/8/1,9,1332,1188,1,2,141,0,0,0,1332,2016-08-01 09:00:00,1191,2016-08-01,2016-08,Monday


この時に全ての列ではなく需要実績と太陽光だけに興味があるなら，以下のようにする。

In [13]:
selected_day="2016-08-01"
display(df_loadokinawa.query('年月日 == @selected_day')[["需要実績","太陽光"]])

,需要実績,太陽光
2928,1008,0
2929,946,0
2930,933,0
2931,925,0
2932,916,0
2933,927,0
2934,955,14
2935,1073,58
2936,1206,81
2937,1332,141


抽出する条件は複数を指定可能である。たとえば，2016年8月のデータのうち，太陽光の出力が200以上のデータを抽出してみる。
複数の条件式を`and`で繋ぐことで「2016年8月のデータ」かつ「太陽光が200以上」のデータが抽出される。

不等号としては`>`，`>=`，`<`，`<=`が使用可能である。条件を「かつ」ではなく「または」でつなげたい場合は`or`を使う。

In [14]:
# 複数の条件で抽出
selected_month="2016-08"
minimum_solar=200
display(df_loadokinawa.query('年月 == @selected_month and 太陽光 >= @minimum_solar'))

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,年月,曜日
3180,2016/8/11,12,1294,1087,1,2,204,0,0,0,1294,2016-08-11 12:00:00,1090,2016-08-11,2016-08,Thursday
3204,2016/8/12,12,1431,1226,1,2,200,0,2,0,1431,2016-08-12 12:00:00,1231,2016-08-12,2016-08,Friday
3228,2016/8/13,12,1312,1098,1,2,211,0,1,0,1312,2016-08-13 12:00:00,1101,2016-08-13,2016-08,Saturday
3229,2016/8/13,13,1324,1113,1,2,207,0,1,0,1324,2016-08-13 13:00:00,1117,2016-08-13,2016-08,Saturday
3251,2016/8/14,11,1257,1052,1,2,201,0,1,0,1257,2016-08-14 11:00:00,1056,2016-08-14,2016-08,Sunday
3252,2016/8/14,12,1277,1064,1,2,209,0,2,0,1277,2016-08-14 12:00:00,1068,2016-08-14,2016-08,Sunday
3253,2016/8/14,13,1281,1073,1,2,202,0,2,0,1281,2016-08-14 13:00:00,1079,2016-08-14,2016-08,Sunday
3371,2016/8/19,11,1401,1195,1,2,203,0,0,0,1401,2016-08-19 11:00:00,1198,2016-08-19,2016-08,Friday
3395,2016/8/20,11,1334,1130,1,2,201,0,0,0,1334,2016-08-20 11:00:00,1133,2016-08-20,2016-08,Saturday
3563,2016/8/27,11,1280,1068,1,2,201,0,9,0,1280,2016-08-27 11:00:00,1079,2016-08-27,2016-08,Saturday


条件式は範囲で指定可能である。たとえば2016年8月1日から2016年8月7日の一週間の11時から13時のデータを表示させてみる。

In [15]:
# 特定の範囲の日付の特定の時間のみ
start_date="2016-08-01"
end_date="2016-08-07"
start_time=11
end_time=13
df_loadokinawa.query('@start_date <= 年月日 <= @end_date and @start_time <= 時刻 <= @end_time')

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,年月,曜日
2939,2016/8/1,11,1407,1220,1,2,185,0,0,0,1407,2016-08-01 11:00:00,1222,2016-08-01,2016-08,Monday
2940,2016/8/1,12,1403,1226,1,2,174,0,0,0,1403,2016-08-01 12:00:00,1229,2016-08-01,2016-08,Monday
2941,2016/8/1,13,1378,1219,1,2,157,0,0,0,1378,2016-08-01 13:00:00,1221,2016-08-01,2016-08,Monday
2963,2016/8/2,11,1403,1231,1,2,170,0,0,0,1403,2016-08-02 11:00:00,1233,2016-08-02,2016-08,Tuesday
2964,2016/8/2,12,1382,1218,1,2,161,0,0,0,1382,2016-08-02 12:00:00,1221,2016-08-02,2016-08,Tuesday
2965,2016/8/2,13,1399,1258,1,2,139,0,0,0,1399,2016-08-02 13:00:00,1260,2016-08-02,2016-08,Tuesday
2987,2016/8/3,11,1426,1234,1,2,189,0,0,0,1426,2016-08-03 11:00:00,1237,2016-08-03,2016-08,Wednesday
2988,2016/8/3,12,1394,1207,1,2,184,0,0,0,1394,2016-08-03 12:00:00,1210,2016-08-03,2016-08,Wednesday
2989,2016/8/3,13,1415,1228,1,2,184,0,0,0,1415,2016-08-03 13:00:00,1231,2016-08-03,2016-08,Wednesday
3011,2016/8/4,11,1327,1240,1,2,84,0,1,0,1327,2016-08-04 11:00:00,1243,2016-08-04,2016-08,Thursday


## Bokehを利用したグラフ描画

### 毎時データのプロット
Pythonには多くのグラフ描画ライブラリがあるが，ここではBokehを利用してグラフを描いてみる。

以下の例は2016年の8月の1時間ごとのグラフを描いている。

まず，二つの変数`start_datetime`と`end_datetime`にグラフに表示する最初の日時と最後の日時を設定している。

次に`p = figure`で図の大枠を準備する。`titile=`は図のタイトルを指定する。`x_axis_type='datetime'`はx軸はdatetime型の値が与えられることを教えている。これを指定しないと横軸がどの日時に対応するのかわからない大きな数字になる。`x_axis_label`と`y_axis_label`はx軸とy軸の説明ラベルを指定する。`width`と`height`は図の大きさを指定する。

次に具体的にデータをプロットする。`p.line`は`p`として準備した図の大枠にデータを折線でプロットする。括弧内の最初に与えるものがx座標データ，次がy座標データである。これらを変更することで異なるデータをプロットできる。

`legend_lable=`でプロットした折線の凡例の説明を指定する。

`color=`で線の色を指定する。このファイルの最上部で`Category10`という3色から10色までのパレット（色の組み合わせ）を`palette`として読み込んでいる。`palette[10][0]`とは10色で構成されるパレットの最初の色である。`palette[10][1]`なら10色パレットの2番目の色となる。

`line_width=`は線幅を指定する。

次に`p.circle`でデータ点に○をプロットする。

同じデータを○と折線で描くので`legend_lable`は折線と同じにしてある。

`fill_color='white'`は白抜きの○でプロットすることを指示している。

最後に`p = figure`で指定した図を実際に描くために`show(p)`を呼び出す。

図の右側にあるアイコンを利用してズームや移動を行うことも出来る。

In [16]:
# 毎時のグラフを描く
#
start_datetime="2016-08-01 00:00:00"
end_datetime="2016-08-31 23:00:00"
p = figure(title="沖縄県の需要実績",x_axis_type='datetime', x_axis_label='日時', y_axis_label='1時間の電力量[MWh]',
           y_range=(0,1.1*df_loadokinawa.query('@start_datetime <= 日時 <= @end_datetime')["需要実績"].max()),
           width=800,height=600)
p.line(df_loadokinawa.query('@start_datetime <= 日時 <= @end_datetime')["日時"],
       df_loadokinawa.query('@start_datetime <= 日時 <= @end_datetime')["需要実績"], legend_label="需要実績", 
       color=palette[10][0],line_width=2)
p.circle(df_loadokinawa.query('@start_datetime <= 日時 <= @end_datetime')["日時"],
       df_loadokinawa.query('@start_datetime <= 日時 <= @end_datetime')["需要実績"], legend_label="需要実績", 
       color=palette[10][0],fill_color='white')

# 図を実際に表示
show(p)

### 毎月データのプロット

毎月のデータも同様にプロットすることができる。

「年月」の列は文字列となっていて先ほどの方法では上手くいかない（先ほどの日時は内部では文字列ではなく数値の扱いである）。bokehでは`p=figure`で図の大枠を作るときに`x_range=`でx軸の範囲を明示的に受け渡すことが必要である。
また，一月分をまとめるとy軸の目盛の数値が大きくなるのでデータを1000で割り，MWhからGWhにしている。

In [17]:
p = figure(title="沖縄県の月間需要実績", x_range=df_loadokinawa_monthly_sum.index.values,x_axis_label='年月', 
           y_range=(0,df_loadokinawa_monthly_sum["需要実績"].max()/1000),y_axis_label='1ヶ月の電力量[GWh]',
           width=800,height=600)
p.line(df_loadokinawa_monthly_sum.index.values,df_loadokinawa_monthly_sum["需要実績"].values/1000,legend_label="需要実績", 
       color=palette[10][0],line_width=2)
p.circle(df_loadokinawa_monthly_sum.index.values,df_loadokinawa_monthly_sum["需要実績"].values/1000,legend_label="需要実績", 
       color=palette[10][0],fill_color='white')
show(p)